# Advent of Code 2015
## In the style of Peter Norvig

First some imports and utility Functions.
Most of Peter's utility functions have moved into <a href=file:///Volumes/generic/apuri/g/misc/python/2015-advent-of-code/util.py> util.py </a>
### author: avp

In [138]:
import time
import itertools
import math
import collections
import re
import operator
import numpy
import sys
import string
import re
import functools
import util as u
import more_itertools as mi
#from fastcore.test import test_eq  # fastcore is library I want to learn

def test_eq(actual,expected):  # glorified assert, but when it fails you get some useful info
    if actual != expected:
        print('actual=',actual)
        print('expected=', expected)
        assert False

def Input(n, dir = 'data'):
        return open(f'{dir}/{n:02d}.txt').read()

def remove_all(elt, xs):
        return [x for x in xs if x != elt]



## <a href="https://adventofcode.com/2015/day/1">Day 1 </a> :  Not quite Lisp

In [114]:
def bracketmap(bracketstr):
        return [1 if c == '(' else -1 for c in bracketstr ]

def level(bracketstr):
        return sum(bracketmap(bracketstr))
    
# tests
test_eq([level(s) for s in '(())  ()()  (((  (()(()(  ))(((((  ())  ))(  )))  )())())'.split()],
       [0, 0, 3, 3, 3, -1, -1, -3, -3])

# puzzle
test_eq(level(mi.first(u.Input(1))), 138)

In [45]:
# Part 2

def steps_to_basement(bracketstr):
        "2020.09.16 1905-2005 - mostly to learn doctest"
        return 2+mi.last(enumerate(itertools.takewhile(lambda x: x >= 0, 
                                                       itertools.accumulate(bracketmap(bracketstr)))),
                         (-1,0))[0] # the [0] picks out the enumeration number 

# tests
test_eq([steps_to_basement(s) for s in '()())  )'.split()], [5, 1])

# puzzle
test_eq(steps_to_basement(Input(1, 'data').strip()), 1771)


## <a href="https://adventofcode.com/2015/day/2"> Day 2 </a>: I Was Told There Would Be No Math

In [92]:
def box_area_to_wrap(dims):
        """
        dims is a tuple of box dimensions (length l, width w, and height h)
        to wrap we need the surface area + slack = area of smallest side
        """
        sides = [x*y for x, y in itertools.combinations(dims, 2)]
        return sum(sides) * 2 + min(sides)

# tests
test_eq([box_area_to_wrap(dims) for dims in [(2,3,4), (1,1,10)]], 
        [58, 43])
 
# puzzle
test_eq(sum(map(box_area_to_wrap, u.Input(2, u.integers))), 1586300)


In [100]:
# Part 2
def ribbon(dims):
        """
        2020.09.16 2005-2045

        A present with dimensions 2x3x4 needs 
        2+2+3+3 = 10 feet of ribbon to wrap the present plus 
        2*3*4 = 24 feet of ribbon for the bow, for a total of 34 feet.
        
        A present with dimensions 1x1x10 needs 
        1+1+1+1 = 4 feet of ribbon to wrap the present plus 
        1*1*10 = 10 feet of ribbon for the bow, for a total of 14 feet.
       """
        dims = sorted(dims)
        return sum(dims[:2]) * 2 + math.prod(dims)
    
# tests
test_eq(list(map(ribbon, [(2,3,4),(1,1,10)])),  [34, 14])

# puzzle
test_eq(sum(ribbon(dims) for dims in u.Input(2, u.integers)), 3737498)


   

## <a href="https://adventofcode.com/2015/day/3"> Day 3 </a>: Perfectly Spherical Houses in a Vacuum

In [130]:
P = complex

def pathremap(path):
        "convert the arrows into 'steps' in the complex plane"
        d = {   '^' : P(0,1), '>' : P(1),
                'v' : P(0,-1), '<': P(-1) }
        return map(d.get, path)

def distinct_points(path):
        "2020.09.16 2045-2105 = 20m"
        return len(set(itertools.accumulate(itertools.chain([P(0)], pathremap(path)))))


test_eq([distinct_points(s) for s in ['>', '^>v<', '^v^v^v^v^v']],
        [2, 4, 2])
test_eq(distinct_points(mi.first(u.Input(3))), 2565)


In [144]:
def two_santas(path):
        "2020.09.15 2105-2135 = 30m"
        paths = mi.distribute(2, pathremap(path))  # one path for each santa
        paths = [itertools.accumulate(itertools.chain([P(0)],path)) for path in paths]
        return len(set(itertools.chain(*paths)))

test_eq([two_santas(s) for s in ['^v', '^>v<', '^v^v^v^v^v']],
    [3, 3, 11])
test_eq(two_santas(mi.first(u.Input(3))), 2639)


## <a href="https://adventofcode.com/2015/day/4"> Day 4 </a>: The Ideal Stocking Stuffer

These can take a /while/ to evaluate 

In [178]:
import hashlib

def functional_find_num(key, nzeros=5):
        # NOT USED : just here to show that sometimes fp seems less clear
        return mi.first(i
            for i, m in enumerate(map(u.compose(hashlib.md5,
                                            str.encode, 
                                            functools.partial(operator.add, key), 
                                                    # prepend key
                                            str),   # convert count to a str
                                  itertools.count()))
                if m.hexdigest()[:nzeros] == '0'*nzeros)

                
def find_num(key, nzeros=5):
        "2020.09.16 2135-2200 = 25m"
        for i in itertools.count():
                s = key+str(i)
                m = hashlib.md5(s.encode())
                if m.hexdigest()[:nzeros] == '0'*nzeros: return i

       
# tests
test_eq(u.mapt(find_num, ['abcdef', 'pqrstuv']),
        (609043, 1048970))

# puzzle
%time test_eq(find_num('iwrupvqb'), 346386)

CPU times: user 605 ms, sys: 1.64 ms, total: 607 ms
Wall time: 609 ms


In [177]:
# takes about 20s on my machine
%time test_eq(find_num('iwrupvqb', 6), 9958218)

CPU times: user 21.4 s, sys: 21 ms, total: 21.4 s
Wall time: 21.5 s


## <a href="https://adventofcode.com/2015/day/5"> Day 5 </a>: Doesn't He Have Intern-Elves For This?


In [196]:
def is_nice(s):
        "2020.09.16 2200-2230 = 30m"
        def n_vowels(s):
                return len([c for c in s if c in 'aeiou'])
        def has_double_letter(s):
                return next((x for x, y in mi.pairwise(s) if x == y), None) != None
        def has_forbiddens(s):
                forbiddens = 'ab cd pq xy'.split()
                return any(bad in s for bad in forbiddens)
        return n_vowels(s) >= 3 and has_double_letter(s) and not has_forbiddens(s)

# tests
test_eq(list(map(is_nice, 'ugknbfddgicrmopn aaa jchzalrnumimnmhp haegwjzuvuyypxyu dvszwmarrgswjxmb'.split())),
        [True, True, False, False, False])

# puzzle
test_eq(mi.quantify(map(is_nice,u.Input(5))), 258)

In [191]:
def is_new_nice(s):
        "2020.09.16 2230-2245    15m"
        def has_double_letter_with_gap(s):
                for i in range(len(s)-2):
                        if s[i] == s[i+2]:
                                return True
                return False
        def has_duplicate_nonoverlapping_pair(s):
                for i in range(len(s)-3):
                        if s[i:i+2] in s[i+2:]:
                                return True
                return False
        return has_duplicate_nonoverlapping_pair(s) and has_double_letter_with_gap(s)

# tests
test_eq(list(map(is_new_nice, 'xyxy qjhvhtzxzqqjkmpb xxyxx uurcxstgmygtbstg ieodomkazucvgmuy'.split())),
        [True, True, True, False, False])
# puzzle
test_eq(mi.quantify(map(is_new_nice,u.Input(5))), 53)

## <a href="https://adventofcode.com/2015/day/6"> Day 6 </a>: Probably a Fire Hazard


In [232]:
def get_cmd(words):
        if words[0] == 'toggle':
                words = ['.'] + list(words)
        _, cmd, tlx, tly, _, brx, bry = words
        tlx, tly, brx, bry = map(int,(tlx, tly, brx, bry))
        return cmd, tlx, tly, brx+1, bry+1

def nlights(instructions):
        # 2020.09.16 2300-2330    30m
        sz = 1000
        L = [False]*sz*sz

        for words in instructions:
                cmd, tlx, tly, brx, bry = get_cmd(words)
                for x in range(tlx, brx):
                        for y in range(tly, bry):
                                pt = x + y*sz
                                if cmd == 'toggle':
                                        L[pt] = not L[pt]
                                else:
                                        L[pt] = cmd == 'on'
        return mi.quantify(L)

#assert brightness(instructions) == 14110788
# puzzle
%time test_eq(nlights(u.Input(6, u.words)), 377891)


CPU times: user 3.67 s, sys: 9.59 ms, total: 3.68 s
Wall time: 3.69 s


In [224]:
# this needs get_cmd from the cell above
def brightness(instructions):
        # 2020.09.16 2330-2350  20m - accidentally dropped +P(1,1)
        sz = 1000
        L = [0]*sz*sz

        for words in instructions:
                cmd, tlx, tly, brx, bry = get_cmd(words)
                for x in range(tlx, brx):
                        for y in range(tly, bry):
                                pt = x + y*sz
                                if cmd == 'on':
                                        L[pt] += 1
                                elif cmd == 'off':
                                        L[pt] = max(0, L[pt] - 1)
                                else:
                                        L[pt] += 2
        return sum(L)


# puzzle
%time test_eq(brightness(u.Input(6, u.words)), 14110788)


CPU times: user 5.84 s, sys: 11.4 ms, total: 5.85 s
Wall time: 5.86 s


## <a href="https://adventofcode.com/2015/day/7"> Day 7 </a>: Some Assembly Required

In [253]:
def get_emulated(instructions, initial_registers={}):
        # 2020.09.17    0815-1030
        #               stuck on numpy.uint16, then introducing "pending" dic

        skipB           = initial_registers != {}
        integer_type    = numpy.uint16
        registers       = initial_registers
        pending         = collections.defaultdict(list)
        letters         = re.compile('^[a-z]+$')
        digits          = re.compile('^[0-9]+$')
        ops             = {     'AND'   : operator.and_,
                                'OR'    : operator.or_,
                                'LSHIFT': operator.lshift,
                                'RSHIFT': operator.rshift,
                                'NOT'   : operator.invert,      }

        def eval(x):
                return registers[x] if letters.match(x) else integer_type(x)

        def is_ready(x):
                return  (letters.match(x) == None or x in registers)

        def process(instruction, skipB):
                words = instruction.split()
                if words[1] == '->':
                        # assignment
                        a, _, tgt = words
                        if not is_ready(a):
                                pending[a].append(instruction)
                                return

                        if not (skipB and tgt == 'b'):
                                registers[tgt] = eval(a)
                elif words[2] == '->':
                        # unary op
                        op, a, _, tgt = words
                        if not is_ready(a):
                                pending[a].append(instruction)
                                return
                        if not (skipB and tgt == 'b'):
                                registers[tgt] = ops[op](eval(a))
                elif words[3] == '->':
                        # binary op
                        a, op, b, _, tgt = words
                        if not is_ready(a):
                                pending[a].append(instruction)
                                return
                        if not is_ready(b):
                                pending[b].append(instruction)
                                return
                        if not (skipB and tgt == 'b'):
                                registers[tgt] = ops[op] (eval(a), eval(b))
                else:
                        print(words)
                        assert False

                # process any instructions pending on tgt being ready
                instructions, pending[tgt] = pending[tgt], []
                for instruction in instructions:
                        process(instruction, skipB)


        for instruction in instructions:
                process(instruction, skipB)

        return registers['a']  # return the value of register 'a'

#instructions = Input(7, 'data').strip().split('\n')
instructions = u.Input(7)

# puzzle - part 1
test_eq(get_emulated(instructions), 3176) 

# puzzle - part 2
test_eq(get_emulated(instructions, { 'b': 3176 }), 14710)


## <a href="https://adventofcode.com/2015/day/8"> Day 8 </a>: Matchsticks

In [259]:
def spacecalc(xs):
        # 2020.09.17    2100-2110       10m
        return sum(len(x) - len(eval(x)) for x in xs)

test_eq(spacecalc(u.Input(8)), 1350)



In [265]:
def space2calc(xs):
        #       looked for built in escape for ages before realizing it is trivial
        def escape(str):
                cs = ['"']
                for c in str:
                        if c in '"\\':
                                cs.append('\\')
                        cs.append(c)
                cs.append('"')
                return ''.join(cs)

        return sum(len(escape(x)) - len(x) for x in xs)

test_eq(space2calc(u.Input(8)), 2085)
 


## <a href="https://adventofcode.com/2015/day/9"> Day 9 </a>: All in a Single Night


In [289]:
def shortest(distances, func = min):
        # 2020.09.17    2135-2215       40m
        # 2020.09.17    2215-2217        2m
        cities = set(itertools.chain(*[[x[0],x[2]] for x in distances]))  # x[0] is from city, x[2] is to city
        distances = dict((tuple(sorted((x[0], x[2]))), int(x[3])) for x in distances)
        routes = list(itertools.permutations(cities))
        return func(sum (map(u.compose(distances.get,   # get distance of ordered pair of cities
                                       tuple,           # distances dict key is a tuple, not a list
                                       sorted),         # order them into a list
                             mi.pairwise(route)         # take each city pair along the route
                        ))                              # total route distance
                    for route in routes
        )


distances = u.Input(9, u.words)

# puzzles
test_eq(shortest(distances), 251)
test_eq(shortest(distances, max), 898)



## <a href="https://adventofcode.com/2015/day/10"> Day 10 </a>: Elves Look, Elves Say
